## Dependencies
* openai

In [1]:
import sys
sys.path.append("../")

To fully run this notebook you need to:

* Clone the GitHub repo 

In [87]:
import csv
import openai
import re
from src.decorators import load_key_secret

In [129]:
path = "/home/propietari/Documents/claus/" # TODO: Adjust to your path
filename = "OpenAI_keys_JordiPlanas.json" # TODO: Adjust to your filename

@load_key_secret(path, filename, "secret")
def return_secret(secret):
    return secret

def create_completion(text_to_complete, number_of_samples, tokens):
    return openai.Completion.create(
        engine = "davinci-instruct-beta",
        prompt = text_to_complete,
        temperature = 1,
        max_tokens = tokens,
#         stop = "\n\n",
        n = number_of_samples #by default it gives you 4 choices so you multiply 4 times n
    )

def import_text_from_file(language):
    path = "../input/"
    if "atal" in language:
        filename = "Chorus_Catalan.txt"
    elif "ngli" in language:
        filename = "Chorus_English.txt"
    file = path + filename
    with open(file) as f:
        lines = f.readlines()
    return lines

def calculate_tokens(text):
    string = ""
    length = 0
    i = 0
    j = 0
    for item in text:
        if j > 0:
            if len(item) < 2:
                length = length + len(string)
                i += 1
                string = ""
            else:
                string = string + item
        j += 1
        
    return(int((length/i)/4))

def create_list_of_prompts(text):
    i = 0
    prompts = []
    string = ""
    for item in text:
        if len(item) > 1:
            if i < 20:
                flag = True
                string = string + item
            else:
                flag = False
                prompts.append(string)
                i = -1
                string = item      
        i += 1
    if flag:
        prompts.append(string)
    return prompts
        

find_digit = re.compile(r'\d')
def filter_response(response, stanza_length, max_verse_length, min_verse_length):
    clean_list = []
    for item in response["choices"]:
        new_text = item["text"]
        new_text = new_text.strip("\n")
        new_text = new_text.replace("\n\n", "\n")
        text_list = new_text.split("\n")
        string = "\n"
        flag = True
        if len(text_list) >= stanza_length:
            for verse in text_list[0:stanza_length]:
                if re.search(r'\d+', verse) is None and len(verse) < max_verse_length and len(verse) > min_verse_length:
                    string = string + verse + "\n"
                else:
                    flag = False
            if flag is True:        
                clean_list.append(string)
    return clean_list
            
    

def show_and_save_responses(language,filtered_response):
    path = "../output/"
    if "atal" in language:
        filename = "Catalan_chorus.txt"
    elif "ngli" in language:
        filename = "English_chorus.txt"
    file = path + filename
    i = 0
    for stanza in filtered_response:
        print(f"*** {i} ***\n{stanza}\n")
        i += 1
        with open(file, 'a+') as f:
            f.write(stanza)

In [4]:
openai.api_key = return_secret()

In [120]:
language = "English" # Select "Catalan" if you want to generate text in catalan

original_text = import_text_from_file(language)

maxim = 0
minim = 50
j = 0
for i,item in enumerate(original_text):
    if j > 1:
        if len(item) > maxim:
            maxim = len(item)
        if len(item) < minim and len(item) > 1:
            minim = len(item)
    j += 1
    
print(f"chorus average number of  tokens:{calculate_tokens(original_text)}, max verse length is:{maxim}, and min verse length is:{minim}")
average_tokens = calculate_tokens(original_text)

chorus average number of  tokens:24, max verse length is:39, and min verse length is:27


In [132]:
output_stanza_length = 4
max_verse_length = 40
min_verse_length = 27
number_of_completions = 20

In [133]:
for text_to_complete in create_list_of_prompts(original_text):
    response = create_completion(text_to_complete, number_of_completions, int(average_tokens * 2.5))
    filtered_response = filter_response(response, output_stanza_length, max_verse_length, min_verse_length)
    show_and_save_responses(language, filtered_response)

*** 0 ***

I feel the wires and I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse
I keep discovering new things


*** 1 ***

I feel the wires, I feel the wires
I feel the wires, I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse


*** 2 ***

I'm finding new phrases, new colors
The sun's a little bit brighter
The day goes on a little longer
I'm reflecting all difficulties




In [131]:
response

<OpenAIObject text_completion id=cmpl-2pLTTCEEdrTi2kyahIc50tk6Q7JwI at 0x7fa9b661a770> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "There will be light, sun will rise\n\"\n\nIts about falling in love and being able to sense the feelings of somebody else's heart.\n\nElectric Youth recorded \"I Feel the Wires\" as their contribution to \"The Hunger Games Mockingjay Part 1 (Original Motion Picture Soundtrack)\" (released November 17, 2014).\n\nThe song has been praised by writers from Pitchfork Media, PopSugar, Bandcamp, Vh1 News, and Beautiful Noise. Raul Paez, writer for Bandcamp, commented, \"I love this band. I love this"
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "text": "I feel the wires, I feel the wires\nI fell in love with the wires\nAnd all of the sweet notes\nI feel the wires, I feel the wires\n\"I Fell In Love With the Wires\" by the Avett Brothers\n\n\"

In [85]:
for item in resultats_per_guardar:
    print(item)


A prop d’una llum intensa
Puja l’esperit en amunt
i la mort cau a terra, morta’.
I accepto tot el que passa


sense que t'hagis adonat
del propòsit que fins ara t'esperava
i es tanquen les portes de foscor
que van obres, camins de llum


Quan ens somiem els dos
som patrimoni il·legit
Ah! no hi ha objecte de l’amor
sinó el món que tenim alçat



In [97]:
for item in resultats_per_guardar:
    with open('../output/Englis_chorus.txt', 'a+') as f:
        f.write(item)
#         write = csv.writer(f)

#         write.writerow(resultats_per_guardar)